In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="llama3-8b-8192",api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000213E21A2C20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000213E21A1960>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

In [5]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
import bs4
loader = WebBaseLoader("https://www.mapcommunications.com/call-center-faq/")
docs = loader.load()
docs

[Document(metadata={'source': 'https://www.mapcommunications.com/call-center-faq/', 'title': '\nWhat Is a Call Center? - Call Center FAQs | MAP Communications', 'description': 'MAP Communications answers the most frequently asked questions about call centers. Find the answers to your questions here!', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\nWhat Is a Call Center? - Call Center FAQs | MAP Communications\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThe First HITRUST Certified Answering Service\n\n MyMAP LOGIN Start my free trial\n\n   \n\n\n\n\nSales: 888-252-6555 Customer Service: 800-627-0114\n\n\nRequest a Free Trial \n\n\n\n Menu \n\nServices\n\nVirtual Receptionist Services\nAnswering Services\nCall Center Services\n\n\nIndustries\n\nSmall Businesses\nMedical\n\nHIPAA Compliance\nA

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitters = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
final_docs = text_splitters.split_documents(docs)

In [8]:
db = Chroma.from_documents(final_docs,embeddings)
retreiver = db.as_retriever()
retreiver


VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000213C915F430>, search_kwargs={})

In [ ]:
system_prompt = (
    """ You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer
    the question. If you don't know the answer, say that you 
    don't know. Use three sentences maximum and keep the 
    answer concise.
    """
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("human","{input}")
]
)

In [10]:
chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retreiver,chain)

In [11]:
response = rag_chain.invoke({"input":"What is an Outbound call?"})
response

{'input': 'What is an Outbound call?',
 'context': [Document(metadata={'description': 'MAP Communications answers the most frequently asked questions about call centers. Find the answers to your questions here!', 'language': 'en-US', 'source': 'https://www.mapcommunications.com/call-center-faq/', 'title': '\nWhat Is a Call Center? - Call Center FAQs | MAP Communications'}, page_content='Our advanced technology and trained agents allow us to deliver efficient and professional support. This inevitably leads to increased efficiency and customer loyalty. We can also provide valuable insights through data analytics, empowering you to make informed decisions and optimize your service strategies for long-term success.\nWhat is an Outbound Call?\nOutbound calls occur when a business or organization initiates a call with a customer or client. For instance, a cable company might dial up a customer who is late on their monthly bill to request payment. Telemarketing is also a common type of outbou

In [12]:
response['answer']

'According to the provided context, an Outbound call occurs when a business or organization initiates a call with a customer or client, such as a cable company calling a customer to request payment or a telemarketing call.'

Adding Chat History and Memory

In [16]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

q_prompt = (""" 
            Given a chat history and the latest user question 
    which might reference context in the chat history,
    formulate a standalone question which can be understood 
    without the chat history. Do NOT answer the question, 
    just reformulate it if needed and otherwise return it as is.
            """)

prompt2 = ChatPromptTemplate.from_messages([
    ("system", q_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

- When I say that q_prompt (used in history_aware_retriever) "reformulates questions for a clearer standalone interpretation if needed," I mean that it is designed to rewrite the user's question in a way that can be understood on its own, without relying on previous messages.

Here’s why this is helpful and what it means in practice:

- Imagine a conversation where the user asks follow-up questions. Often, these questions are not fully self-contained. For example:

- Initial Question: “What is an Inbound call center?”
Follow-up Question: “Tell me more about it?”
The second question, "Tell me more about it?" lacks context by itself. On its own, it’s unclear what "it" refers to. So, before retrieving relevant documents or responding, we need to rewrite or reformulate it into a standalone question, like:

- Reformulated Question: “Tell me more about an Inbound call center.”
This reformulation makes the follow-up question complete and independent, helping the retrieval system fetch the right information. q_prompt instructs the model to perform this reformulation by considering both the new question and the past chat history.

- Why history_aware_retriever Uses q_prompt
When history_aware_retriever receives a follow-up question, it:

- Checks the chat history to identify the context.
Reformulates the question if necessary, using q_prompt, so the question is self-contained.
Uses this clearer question to retrieve the most relevant information.
This way, history_aware_retriever ensures that every query is fully understandable, even if the user’s input depends on prior messages.

In [17]:
history_aware_retriever = create_history_aware_retriever(llm,retreiver,prompt2)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000213C915F430>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [19]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [20]:
chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,chain)

In [21]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What is an Inbound call center?"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Tell me more about it?"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

An inbound call center is a centralized hub where trained agents answer phone calls from customers, clients, or other individuals, providing support, answering questions, and resolving issues.


In [22]:
chat_history

[HumanMessage(content='What is an Inbound call center?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='An inbound call center is a centralized hub where trained agents answer phone calls from customers, clients, or other individuals, providing support, answering questions, and resolving issues. This type of call center does not initiate calls, but rather answers incoming calls from customers who are seeking assistance.', additional_kwargs={}, response_metadata={})]

In [23]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

This code introduces a mechanism for managing ongoing chat history sessions, allowing the bot to remember past conversations across sessions. Here’s an explanation of each part and why it may be necessary after the earlier setup:

Key Components of This Code

- ChatMessageHistory and store Dictionary:
ChatMessageHistory manages and stores messages for individual chat sessions.
store is a dictionary that keeps track of chat history for each unique session (e.g., session_id like "abc123").
get_session_history is a helper function that checks if a chat session already exists in store. If it doesn’t, it creates a new history for that session.

- RunnableWithMessageHistory:
RunnableWithMessageHistory wraps the existing rag_chain so that each call maintains and references a session-specific chat history.
This setup ensures that for any given session, all previous user inputs and AI responses are preserved, allowing the bot to refer back to prior interactions.

- Explanation of invoke with Session ID:
conversational_rag_chain.invoke is called with both a user input and a configuration dictionary that includes a session_id (e.g., "abc123").
This session_id helps manage and recall chat history specific to the session so that context isn’t lost between interactions.
Why RunnableWithMessageHistory is Useful
This approach is particularly helpful if you want your chatbot to handle multiple conversations over time, especially in applications where each user might interact with the chatbot across multiple sessions. By tracking session_id, it keeps each 
user's history separate, ensuring that:

- Users can resume conversations from where they left off.
- Context is maintained across multiple messages, so follow-up questions and responses stay relevant.

Flow of the Code
- Session Initialization: When conversational_rag_chain.invoke is called for the first time with session_id = "abc123", get_session_history checks if "abc123" exists in store. If not, it initializes a new ChatMessageHistory.
Message Handling: Each time a new question is asked, RunnableWithMessageHistory appends it to the session’s chat history, while also accessing past messages from this session.
Answer Retrieval and History Update: After getting the answer, it’s stored back into the session-specific chat history for future reference. This way, subsequent questions are understood in light of past interactions within the same session.
Is It Necessary?
This setup is essential if:

You want the bot to track long-term context and conversation history for each user individually.
Your application needs to handle distinct sessions where different users or conversations may overlap in time.
If your use case involves multiple sessions and requires remembering conversations, then RunnableWithMessageHistory is highly beneficial as it brings session persistence to your bot. Without this, each interaction would be treated independently, losing continuity.

In [24]:
conversational_rag_chain.invoke(
    {"input": "What is the Difference Between Inbound and Outbound Call Center Services?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'The key difference between inbound and outbound call center services is who initiates the call. Inbound call centers handle calls made by consumers, while outbound call centers have agents dialing consumers.'

In [25]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is the Difference Between Inbound and Outbound Call Center Services?', additional_kwargs={}, response_metadata={}), AIMessage(content='The key difference between inbound and outbound call center services is who initiates the call. Inbound call centers handle calls made by consumers, while outbound call centers have agents dialing consumers.', additional_kwargs={}, response_metadata={})])}

You're absolutely right, and I apologize for the confusion in my earlier response! Let me clarify the concept again with the correct understanding.

Key Points:
MessagesPlaceholder is indeed used to store and pass the chat history for each request. This means that, even without RunnableWithMessageHistory, you can still have context from previous messages when you pass the chat history manually in each request.

The absence of RunnableWithMessageHistory does not necessarily mean that the chat history is lost between requests. The context (chat history) would still be passed explicitly as part of the input using MessagesPlaceholder each time.

So, What's Different Without RunnableWithMessageHistory?
Without RunnableWithMessageHistory, you would manually manage the chat history in your code. Here's the difference:

With RunnableWithMessageHistory: The session history is automatically stored and retrieved across multiple requests without you needing to manage it manually. It simplifies the process because the session is handled internally by Langchain, and you don’t have to worry about storing and fetching the conversation history explicitly. It works well when you want the bot to "remember" past conversations across multiple interactions without having to store or manage history on your own.

Without RunnableWithMessageHistory: You need to manually manage the chat history in your code. Each time the user asks a new question, you need to pass the entire chat history (including past questions and answers) to the MessagesPlaceholder. This means you are still able to maintain the context, but the responsibility for storing and maintaining the history falls on you. If you don’t store the history yourself (like in a list or a database), you might lose it across requests.

Yes, exactly! You don’t need to use both methods at the same time.

Key takeaway:
RunnableWithMessageHistory is a more automated approach for handling chat history, so if you're using it, there's no need to manually handle the chat history with MessagesPlaceholder.
MessagesPlaceholder can be used independently for managing the chat history if you prefer to manually pass and track the history between requests.
Both approaches serve the same purpose of maintaining context across conversations, but you only need to choose one method based on your use case. Using both simultaneously is redundant and unnecessary.